# Processing

In this step we generate sparse matrix, general information and ICD9 codes for each patient.

In [30]:
from __future__ import print_function

import psycopg2
import datetime
import sys
from operator import itemgetter, attrgetter, methodcaller
import numpy as np
import itertools
import os.path
import matplotlib.pyplot as plt
import math
from multiprocessing import Pool, cpu_count
import re
import traceback
import shutil
from collections import OrderedDict

from utils import getConnection
from utils import parseUnitsMap
from utils import parseNum
from utils import sparsify

# %matplotlib inline

## Set unit conversion map

We manually set rules for unit conversion and store them in file './config/unitsmap.unit'. The file format is as following:

```
tablename:[name of table in database]
[itemid1],[target unit1],[unit11:ratio11],[unit12:ratio12],...,[unit1n:ratio1n]
...
[itemidn],[target unitn],[unitn1:ration1],[unitn2:ration2],...,[unitnn:rationn]
```

The ratio is set using the following rule: $\mathrm{unit1}*\mathrm{ratio1}=\mathrm{unit2}*\mathrm{ratio2}=...=\mathrm{unitn}*\mathrm{ration}$. For example, one row in the file could be: `227114,mg,mg:1,mcg:1000`. It means that $1\mathrm{mg}=1000\mathrm{mcg}$.

In [31]:
conn = getConnection()
UNITSMAP = parseUnitsMap()

# Set indices for chartevents table

We need to add hadm_id as indices to speed up the query. By default it is not added. Thanks for the help from Weijing Tang@UMich!

You might need to run `grant postgres to <your username>;` before building indices. https://stackoverflow.com/questions/28584640/postgresql-error-must-be-owner-of-relation-when-changing-a-owner-object/28586288

In [33]:
conn = getConnection()
cur = conn.cursor()
indicescomm = '''set search_path to mimiciii; DROP INDEX IF EXISTS chartevents_{0}_idx02;
CREATE INDEX chartevents_{0}_idx02 ON chartevents_{0} (hadm_id);'''
for t in range(1, 18):
    sql = indicescomm.format(t)
    #print("SQL cmd : ",sql)
    cur.execute(sql)
    conn.commit()

KeyboardInterrupt: 

In [29]:
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True)

_adm = np.load('res/admission_ids.npy').tolist()
admission_ids = _adm['admission_ids']
admission_ids_txt = _adm['admission_ids_txt']

_adm_first = np.load('res/admission_first_ids.npy').tolist()
admission_first_ids = _adm['admission_ids']

TypeError: <lambda>() got multiple values for keyword argument 'allow_pickle'

In [5]:
v = np.load('res/filtered_input.npy').tolist()
valid_input = v['id']
valid_input_unit = v['unit']

v = np.load('res/filtered_output.npy').tolist()
valid_output = v['id']

v = np.load('res/filtered_chart.npy').tolist()
valid_chart = v['id']
valid_chart_unit = v['unit']

v = np.load('res/filtered_chart_num.npy').tolist()
valid_chart_num = v['id']
valid_chart_num_unit = v['unit']

v = np.load('res/filtered_chart_cate.npy').tolist()
valid_chart_cate = v['id']

v = np.load('res/filtered_chart_ratio.npy').tolist()
valid_chart_ratio = v['id']

v = np.load('res/filtered_lab.npy').tolist()
valid_lab = v['id']
valid_lab_unit = v['unit']

v = np.load('res/filtered_lab_num.npy').tolist()
valid_lab_num = v['id']
valid_lab_num_unit = v['unit']

v = np.load('res/filtered_lab_cate.npy').tolist()
valid_lab_cate = v['id']

v = np.load('res/filtered_lab_ratio.npy').tolist()
valid_lab_ratio = v['id']

v = np.load('res/filtered_microbio.npy').tolist()
valid_microbio = v['id']

v = np.load('res/filtered_prescript.npy').tolist()
valid_prescript = v['id']
valid_prescript_unit = v['unit']

allids = valid_input+valid_output+valid_chart+valid_chart_num+valid_chart_cate+valid_chart_ratio+valid_chart_ratio+valid_lab+valid_lab_num+valid_lab_cate+valid_lab_ratio+valid_lab_ratio+valid_microbio+valid_prescript
print(len(allids), len(set(allids)))

23634 23533


## Create temporary tables for accelerating the query

In [6]:
# put valid ids into database
conn = getConnection()
cur = conn.cursor()
for itemidlist, itemidlistname in zip([valid_input, valid_output, valid_chart, valid_chart_num, valid_chart_cate, valid_chart_ratio, valid_lab, valid_lab_num, valid_lab_cate, valid_lab_ratio], 'valid_input, valid_output, valid_chart, valid_chart_num, valid_chart_cate, valid_chart_ratio, valid_lab, valid_lab_num, valid_lab_cate, valid_lab_ratio'.replace(' ', '').split(',')):
    sql = 'drop table if exists mengcztemp_itemids_{0}'.format(itemidlistname)
    cur.execute(sql)
    conn.commit()
    sql = 'create table if not exists mengcztemp_itemids_{0} (\
    itemid serial PRIMARY KEY \
    )'.format(itemidlistname)
    cur.execute(sql)
    conn.commit()
    for itemid in itemidlist:
        sql = 'insert into mengcztemp_itemids_{0} (itemid) values ({1})'.format(itemidlistname, itemid)
        cur.execute(sql)
    conn.commit()
    sql = 'select * from mengcztemp_itemids_{0} limit 100'.format(itemidlistname)
    cur.execute(sql)
    res = cur.fetchall()
#     print(res)

In [7]:
cur = conn.cursor()
sql = 'drop table if exists mengcztemp_itemids_{0}'.format('valid_prescript')
cur.execute(sql)
conn.commit()
sql = 'create table if not exists mengcztemp_itemids_{0} (\
    itemid varchar(255) PRIMARY KEY \
    )'.format('valid_prescript')
cur.execute(sql)
conn.commit()
for itemid in valid_prescript:
    sql = 'insert into mengcztemp_itemids_{0} (itemid) values (\'{1}\')'.format('valid_prescript', itemid)
    cur.execute(sql)
conn.commit()
sql = 'select * from mengcztemp_itemids_{0} limit 100'.format('valid_prescript')
cur.execute(sql)
res = cur.fetchall()
print(res, len(res), len(valid_prescript))

[('NEOVIDAYLIN',), ('PRED20',), ('PROP100IG',), ('DEXA0.5L',), ('DOPA400D',), ('IVIGBTL',), ('METH2I',), ('ACETSOLN',), ('APRE125',), ('SENOKOTS',), ('MORP4I',), ('D12.5W250I',), ('SITA100',), ('AA5D151000I',), ('CHLO250',), ('NEBU1000I',), ('TRIM200R',), ('BENGAY2O',), ('PROP225',), ('CAMP200I',), ('RABI10I',), ('ATOR80',), ('CHLORO30',), ('DARV100',), ('FULV250IND',), ('TEGR200',), ('MYCO500I',), ('APRA5OS',), ('ESTR3',), ('SCOP4I',), ('DIG5P',), ('MESA1000R',), ('AMIT25',), ('PROC100I',), ('ADEN90I',), ('SUNI50',), ('F4B1ED',), ('SILD25',), ('METO10I',), ('MAXISES',), ('PERM5T',), ('METH50L',), ('TERAZ1',), ('BENA10',), ('MPRED5I',), ('AGGRCAP',), ('CLOM50',), ('AMIN5',), ('PRISMAK5000',), ('FENT50I',), ('FOLI1',), ('CEFU250',), ('ASCO250I',), ('CALC100SI',), ('FLUT110HFA',), ('PERA200I',), ('TROP5ES',), ('MYCO200/1L',), ('AUGM875L',), ('CIDO1P',), ('BISA5',), ('ALTE20I',), ('TAMS.4',), ('ORBE5T',), ('VANC.125L',), ('AMLO25',), ('NORT50',), ('PHEN50',), ('QUET50XR',), ('CHLO500',), 

In [8]:
print('len(valid_input) = '+ str(len(valid_input)))
print('len(valid_output) = '+ str(len(valid_output)))
print ('len(valid_chart) = '+ str(len(valid_chart)))
print('len(valid_chart_num) = '+ str(len(valid_chart_num)))
print('len(valid_chart_cate) = '+ str(len(valid_chart_cate)))
print('len(valid_chart_ratio) = '+ str(len(valid_chart_ratio)))
print('len(valid_lab) = '+ str(len(valid_lab)))
print('len(valid_lab_num) = '+ str(len(valid_lab_num)))
print('len(valid_lab_cate) = '+ str(len(valid_lab_cate)))
print('len(valid_lab_ratio) = '+ str(len(valid_lab_ratio)))
print('len(valid_microbio) = '+ str(len(valid_microbio)))
print('len(valid_prescript) = '+ str(len(valid_prescript)))
print('\nlen(allids) = '+ str(len(allids)))

len(valid_input) = 3210
len(valid_output) = 1155
len(valid_chart) = 2644
len(valid_chart_num) = 313
len(valid_chart_cate) = 3406
len(valid_chart_ratio) = 100
len(valid_lab) = 372
len(valid_lab_num) = 113
len(valid_lab_cate) = 224
len(valid_lab_ratio) = 1
len(valid_microbio) = 9154
len(valid_prescript) = 2841

len(allids) = 23634


In [9]:
# map itemids to [0..n] column
index = 0
map_itemid_index = {}
allitem = allids
allitem_unit = valid_input_unit + ['NOCHECK'] * len(valid_output) + valid_chart_unit + valid_chart_num_unit + ['NOCHECK'] * len(valid_chart_cate) + ['NOCHECK'] * 2 * len(valid_chart_ratio) + valid_lab_unit + valid_lab_num_unit+ ['NOCHECK']*len(valid_lab_cate)+ ['NOCHECK'] * 2 * len(valid_lab_ratio)+['NOCHECK']*len(valid_microbio) + valid_prescript_unit
for i in range(len(allitem_unit)):
    allitem_unit[i] = allitem_unit[i].replace(' ','').lower()
assert len(allitem) == len(allitem_unit)
for ai in allitem:
    if ai not in map_itemid_index.keys():
        map_itemid_index[ai] = [index]
    else:
        map_itemid_index[ai].append(index)
    index+=1
# print(map_itemid_index)
print(len(map_itemid_index))
np.save('res/map_itemid_index.npy', map_itemid_index)

23533


## Map strings in categorical features to integers and store them to a file

In [10]:
catedict = {}

for i in valid_chart_cate:
    cur = conn.cursor()
    cur.execute('SELECT distinct value FROM mimiciii.chartevents WHERE itemid = '+ str(i) +' and hadm_id in (select * from admission_ids)')
    distinctval = cur.fetchall()
    mapping = {};
    ct = 1;
    for d in distinctval:
        mapping[d[0]]= ct;
        ct+=1;
    catedict[i] = mapping
    print(i)
    
for i in valid_lab_cate:
    cur = conn.cursor()
    cur.execute('SELECT distinct value FROM mimiciii.labevents WHERE itemid = '+ str(i) +' and hadm_id in (select * from admission_ids)')
    distinctval = cur.fetchall()
    mapping = {};
    ct = 1;
    for d in distinctval:
        mapping[d[0]]= ct;
        ct+=1;
    catedict[i] = mapping
    print(i)
    
np.save('res/catedict.npy', catedict)
print('saved!')

1489
223917
4790
5761
5843
5729
5468
5259
4326
2466
4321
6375
8178
576
4683
4976
5478
2784
663
7090
5722
1811
2469
224987
6035
161
4813
3202
224424
7499
1595
3097
8428
4519
4289
1422
2702
3071
2404
57
8457
5604
8499
5719
6922
5644
357
7976
2366
6178
266
4669
225341
6258
5461
366
91
5869
223936
6458
1484
224740
6835
224019
4643
8458
1602
8443
227755
227612
224735
6219
5763
4689
224121
7890
6183
4579
8452
1603
227958
8194
227854
2140
1967
355
3442
6095
6084
5249
4773
6090
6015
5986
453
2099
228199
8516
3530
225018
226130
224977
227832
3463
1474
667
225339
5194
6240
223983
6762
4645
382
6204
2579
378
8475
224640
3378
227591
224012
224423
4893
8374
3671
224904
8138
7691
44
8270
4719
2150
6180
1022
944
8367
4371
8416
4430
551
7292
224327
224985
6086
6091
224894
7141
8366
5873
209
7520
6256
1747
1748
972
224091
5205
3374
227760
4702
5442
224442
7985
4606
227567
4391
4948
4620
4587
6210
4548
227620
4705
1048
360
99
629
1215
224814
8407
4225
5839
5049
4911
2866
8522
5924
8453
224113
7108
22820

8473
4824
432
1061
76
3684
227835
4513
225014
4437
5742
346
68
80
225007
5858
6288
224369
224999
8419
2860
3423
5812
7074
1892
262
6296
225022
223955
6112
3343
399
238
547
227241
7138
124
224032
3372
1623
4744
5473
4952
6510
7871
5743
3365
5293
3551
225000
2834
8137
225099
3694
227652
6009
5098
4354
224023
7948
5260
224457
5982
7258
1116
2020
2862
6143
224909
5962
4555
616
4282
6622
5048
8019
516
542
395
8373
5561
6562
5853
163
352
224915
224991
5110
3353
1512
362
8519
688
2882
111
5717
224906
4524
5500
655
223988
3408
225987
225045
3544
1225
150
7043
2304
228447
3283
8470
5831
8523
4213
4858
8439
7029
224888
4958
2359
1125
148
6062
7470
8230
6430
224101
224085
350
6108
8034
8456
6002
3817
5590
227943
224087
8058
5311
5752
6517
227607
224062
7032
7301
1493
5733
940
2855
1963
4968
2631
4791
228404
4407
6528
224692
632
7174
225023
6939
4417
5141
224844
6030
224000
224170
8436
160
7281
644
4518
5769
6257
1644
5606
4696
5875
8437
5750
8247
1606
3476
1312
5692
2684
226168
223920
4271
6113
2

224391
227764
225182
5167
5470
4217
522
674
401
6244
1483
8071
1094
4436
1948
1110
223059
1743
3118
4486
3640
4355
85
6270
2848
2788
223999
4247
7688
3216
226480
1027
8379
4412
1832
7380
151
1346
7953
4661
620
7383
224850
6034
4523
4467
2066
6852
2086
224893
783
5946
4428
1739
464
225231
1980
6895
284
2794
4729
227638
4389
2449
7058
1019
7166
224731
136
224109
1459
7547
6304
4311
5214
5789
3369
224933
4400
5971
5891
223782
5941
403
7875
5770
1643
6287
227846
6203
6166
1339
224114
5032
3447
1855
5913
552
7856
3628
228405
4381
345
8510
5693
8382
4716
225956
197
4905
5245
7747
6533
707
2802
4365
5957
5788
6162
5153
228413
227836
5785
227966
225046
4850
6347
1069
7286
1043
227648
228300
225952
691
5265
697
2604
5747
4990
6556
223946
104
224829
1591
397
8489
2569
227929
427
6190
224865
6515
228401
1652
227517
2693
5966
3139
5548
4855
227616
8434
253
692
7375
4810
3208
5128
3567
227487
6025
8421
223792
7078
8093
8177
224064
603
383
8398
3418
4604
3484
225388
5462
227759
225746
4823
3089
6111

In [11]:
catedict = np.load('res/catedict.npy').tolist()
# print(catedict)

In [12]:
def convert_units(unitmap, src_unit, dst_unit, src_value, f):
    try:
        src_ratio = unitmap['umap'][src_unit]
        dst_ratio = unitmap['umap'][dst_unit]
    except:
        print('converterror: ', unitmap, src_unit, dst_unit, src_value, file=f)
        return None
    if src_ratio == 0:
        return None
    else:
        return float(src_value) / src_ratio * dst_ratio

## Processing inputevents

1. Discard records without starttime.
2. Discard records without amount.
3. If the itemid matches manual rules, convert it according to the rule; else only keep the main unit and discard all other records not having the main unit.

In [13]:
def processing_inputevents(aid, admittime, conn, f):
    cur = conn.cursor()
    sql = '''select starttime, itemid, amount, amountuom from mimiciii.inputevents_mv where amount>0 and hadm_id={0} and itemid in (select * from mengcztemp_itemids_valid_input)
union all
select charttime as starttime, itemid, amount, amountuom from mimiciii.inputevents_cv where amount>0 and hadm_id={0} and itemid in (select * from mengcztemp_itemids_valid_input)'''.format(aid)
    cur.execute(sql)
    inputevents = cur.fetchall()
    inputevents_wholedata = []
    for ie in inputevents:
        starttime, itemid, amount, amountuom = ie[0], ie[1], ie[2], ie[3]
        # discard values with no starttime
        if starttime is None:
            print('no starttime: ', ie, file=f)
            continue
        # discard values with no amount
        if amount is None:
            print('no amount: ', ie, file=f)
            continue
        # convert units...
        amountuom = amountuom.replace(' ', '').lower()
        unitmap = UNITSMAP['inputevents']
        mainunit = allitem_unit[map_itemid_index[itemid][0]]
        if itemid in unitmap.keys():
            dst_value = convert_units(unitmap[itemid], amountuom, mainunit, amount, f)
        else:
            if amountuom == mainunit:
                dst_value = float(amount)
            else:
                dst_value = None
        if dst_value is None:
            print('not convertible: ', ie, file=f)
            continue
        inputevents_wholedata.append(['ie', (starttime - admittime).total_seconds(), [starttime, itemid, dst_value, mainunit]])
    return inputevents_wholedata

# processing_inputevents(184834, datetime.datetime.now(), getConnection(), sys.stdout)

## Processing outputevents

We only need to discard records without starttime or value.

In [14]:
def processing_outputevents(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value FROM mimiciii.outputevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_output)')
    outputevents = cur.fetchall()
    wholedata = []
    for oe in outputevents : 
        #check date
        if(oe[0] == None):
            print('no starttime', oe, file=f)
            continue
        #discard none value
        if(oe[2] == None):
            print('no value', oe, file=f)
            continue
        # no need to check unit all is mL
        oe = list(oe)
        oe.append('ml')
        wholedata.append(['oe',(oe[0]-admittime).total_seconds(),oe])
    return wholedata

# processing_outputevents(184834, datetime.datetime.now(), getConnection(), f=sys.stdout)

## Processing chartevents/labevnets

1. Discard records without starttime or value/valueuom.
2. Process 4 kinds of chartevents/labevents features separately.
    1. valid numerical features(numerical features not needing parsing): only need to convert units.
    2. categorical features: only need to map strings to integers.
    3. possible numerical features(values need parsing):
        1. parse values
        2. convert units
    4. ratio features: store two numbers in the ratio separately.

In [15]:
def processing_chartevents(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,valuenum,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_chart)')
    chartevents = cur.fetchall()
    wholedata = []
    for ce in chartevents:     
        #check date
        if ce[0] is None:
            print('no starttime: ', ce, file=f)
            continue
    
        #discard none value and none valueuom
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
    
        #tuple to list
        ce = list(ce)

        # convert units...
        if ce[3] is None:
            ce[3] = ''
        itemid, valuenum, valueuom = ce[1], ce[2], ce[3]
        valueuom = valueuom.replace(' ', '').lower()
        unitmap = UNITSMAP['chartevents']
        mainunit = allitem_unit[map_itemid_index[itemid][0]]
        ce[3] = mainunit
        if itemid in unitmap.keys():
            dst_value = convert_units(unitmap[itemid], valueuom, mainunit, valuenum, f)
        else:
            if valueuom == mainunit or valueuom == '':
                dst_value = float(valuenum)
            else:
                dst_value = None
        ce[2] = dst_value
        
        #discard none value
        if(ce[2] == None):
            print('not convertible: ', ce, file=f)
            continue
        
        wholedata.append(['ce',(ce[0]-admittime).total_seconds(),list(ce)])
    return wholedata

def processing_chartevents_cate(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_chart_cate)')
    chartevents = cur.fetchall()
    wholedata = []
    for ce in chartevents: 
        #check date
        if ce[0] is None:
            print('no starttime: ', ce, file=f)
            continue
            
        #discard none value
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
        
        #tuple to list
        ce = list(ce)
        
        #map to num
        ce[2] = catedict[ce[1]][ce[2]]
        if ce[2] is None:
            continue
        wholedata.append(['cecate',(ce[0]-admittime).total_seconds(),list(ce)])
    return wholedata
        
def processing_chartevents_num(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_chart_num)')
    chartevents = cur.fetchall()
    wholedata = []
    # parse numerical values
    for ce in chartevents:
        #tuple to list
        ce = list(ce)
        
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
        ce2res = parseNum(ce[2])
        
        #check date
        if(ce[0] == None):
            print('no starttime: ', ce, file=f)
            continue

        #discard none value
        if(ce2res == None):
#             writeline(f,'lenum None value :' + str(le))
            print('not parsed: ', ce, file=f)
            continue
        else:
            ce[2] = ce2res
    
        #check unit
        unitmap = UNITSMAP['chartevents']
        if ce[3] is None:
            ce[3] = ''
        currentunit = ce[3].replace(' ','').replace('<','').replace('>','').replace('=','').lower()
        mainunit = allitem_unit[map_itemid_index[ce[1]][0]]
        if( currentunit == mainunit or currentunit == ''):
            pass
        else:    
            if ce[1] in unitmap.keys():
                ce[2] = convert_units(unitmap[ce[1]], currentunit, mainunit, ce[2], f)
            else:
                if currentunit != mainunit:
                    ce[2] = None
        
        #discard none value
        if(ce[2] == None):
            print('not convertible: ', ce, file=f)
            continue
        
        wholedata.append(['cenum',(ce[0]-admittime).total_seconds(),list(ce)])
    return wholedata

def processing_chartevents_ratio(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.chartevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_chart_ratio)')
    chartevents = cur.fetchall()
    wholedata = []
    for ce in chartevents:
        ce = list(ce)
        if ce[0] is None:
            print('no starttime: ', ce, file=f)
            continue
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
        try:
            fs = ce[2].split('/')
            f1, f2 = fs[0], fs[1]
            if f1 != '':
                ce[2] = float(f1)
                wholedata.append(['leratio_1', (ce[0] - admittime).total_seconds(), list(ce)])
            if f2 != '':
                ce[2] = float(f2)
                wholedata.append(['leratio_2', (ce[0] - admittime).total_seconds(), list(ce)])
        except:
            print('not parsed: ', ce, file=f)
            continue
    return wholedata
        
# processing_chartevents_ratio(136796, datetime.datetime.now(), getConnection(), f=sys.stdout)

In [16]:
def processing_labevents(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,valuenum,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_lab)')
    chartevents = cur.fetchall()
    wholedata = []
    for ce in chartevents:     
        #check date
        if ce[0] is None:
            print('no starttime: ', ce, file=f)
            continue
    
        #discard none value
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
    
        #tuple to list
        ce = list(ce)

        # convert units...
        if ce[3] is None:
            ce[3] = ''
        itemid, valuenum, valueuom = ce[1], ce[2], ce[3]
        valueuom = valueuom.replace(' ','').replace('<','').replace('>','').replace('=','').lower()
        unitmap = UNITSMAP['labevents']
        mainunit = allitem_unit[map_itemid_index[itemid][0]]
        ce[3] = mainunit
        if itemid in unitmap.keys():
            dst_value = convert_units(unitmap[itemid], valueuom, mainunit, valuenum, f)
        else:
            if valueuom == mainunit or valueuom == '':
                dst_value = float(valuenum)
            else:
                dst_value = None
        ce[2] = dst_value
        
        #discard none value
        if ce[2] is None:
            print('not convertible: ', ce, file=f)
            continue
        
        wholedata.append(['le',(ce[0]-admittime).total_seconds(),list(ce)])
    return wholedata

def processing_labevents_cate(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_lab_cate)')
    labevents = cur.fetchall()
    wholedata = []
    for le in labevents: 
        #check date
        if le[0] is None:
            print('no starttime: ', le, file=f)
            continue
            
        #discard none value
        if le[2] is None:
            print('no value: ', le, file=f)
            continue
        
        #tuple to list
        le = list(le)
        
        #map to num
        le[2] = catedict[le[1]][le[2]]
        if le[2] is None:
            continue
        wholedata.append(['lecate',(le[0]-admittime).total_seconds(),list(le)])
    return wholedata

def processing_labevents_num(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_lab_num)')
    labevents = cur.fetchall()
    wholedata = []
    # parse numerical values
    for le in labevents:
        #tuple to list
        le = list(le)
        
        if le[0] is None:
            print('no starttime', le, file=f)
            continue
        
        if le[2] is None:
            print('no value: ', le, file=f)
            continue
        
        #translate values
        le2res = parseNum(le[2])

        #discard none value
        if le2res is None:
            print('not parsed: ', le, file=f)
            continue
        else:
            le[2] = le2res
    
        #check unit
        unitmap = UNITSMAP['labevents']
        if le[3] is None:
            le[3] = ''
        currentunit = le[3].replace(' ','').replace('<','').replace('>','').replace('=','').lower()
        mainunit = allitem_unit[map_itemid_index[le[1]][0]]
        if( currentunit == mainunit or currentunit == ''):
            pass
        else:    
            if le[1] in unitmap.keys():
                le[2] = convert_units(unitmap[le[1]], currentunit, mainunit, le[2], f)
            else:
                if currentunit != mainunit:
                    le[2] = None
        
        #discard none value
        if(le[2] == None):
            print('not convertible: ', le, file=f)
            continue
        
        wholedata.append(['lenum',(le[0]-admittime).total_seconds(),list(le)])
    return wholedata

def processing_labevents_ratio(aid, admittime, conn, f):
    cur = conn.cursor()
    cur.execute('SELECT charttime,itemid,value,valueuom FROM mimiciii.labevents WHERE hadm_id = '+str(aid)+' and itemid in (select * from mengcztemp_itemids_valid_lab_ratio)')
    chartevents = cur.fetchall()
    wholedata = []
    for ce in chartevents:
        ce = list(ce)
        if ce[0] is None:
            print('no starttime: ', ce, file=f)
        if ce[2] is None:
            print('no value: ', ce, file=f)
            continue
        try:
            fs = ce[2].split('/')
            f1, f2 = fs[0], fs[1]
            if f1 != '':
                ce[2] = float(f1)
                wholedata.append(['leratio_1', (ce[0] - admittime).total_seconds(), list(ce)])
            if f2 != '':
                ce[2] = float(f2)
                wholedata.append(['leratio_2', (ce[0] - admittime).total_seconds(), list(ce)])
        except:
            print('not parsed: ', ce, file=f)
            continue
    return wholedata

# processing_labevents_ratio(145834, datetime.datetime.now(), getConnection(), sys.stdout)

## Process microbiologyevents

1. Discard records without starttime.
2. Parse dose value in dilution_text. Values which can be parsed only contains '<'/'>'/'=' and numbers.

In [17]:
def processing_microbiologyevents(aid, admittime, conn, f):
    wholedata = []
    for m in valid_microbio:
        cur = conn.cursor()
        sql = 'SELECT charttime,(spec_itemid,org_itemid,ab_itemid),dilution_text,\'uom\' FROM mimiciii.microbiologyevents WHERE hadm_id = '+str(aid)
        m = list(map(str, m))
        if(m[0] == 'None'):
            sql += ' and spec_itemid is null'
        else:
            sql += ' and spec_itemid = '+m[0]
    
        if(m[1] == 'None'):
            sql += ' and org_itemid is null'
        else:
            sql += ' and org_itemid = '+m[1]
        
        if(m[2] == 'None'):
            sql += ' and ab_itemid is null'
        else:
            sql += ' and ab_itemid = '+m[2]

        cur.execute(sql)
        microevents = cur.fetchall()
        for me in microevents: 
            me = list(me)
            for x in range(len(m)):
                try:
                    m[x] = int(m[x])
                except:
                    m[x] = None
            me[1] = tuple(m)
            #checkdate
            if(me[0] == None):
#                 writeline(f,'me date 0 : '+ " : "+str(me))
                print('no starttime: ', me, file=f)
                continue;

            #discard none value
            if(me[2] == None):
#                 writeline(f,'MICRO None value :' + str(me))
                print('no value: ', me, file=f)
                continue;
            
            #tuple to list
            me = list(me)
            
            #formatting 
            dose = me[2]
            dose = dose.replace('<','').replace('>','').replace('=','')
            numVal = None;
            if(dose == ''):
#                 writeline(f,'me parse fail : '+ " : "+str(me))
                print('not parsed: ', me, file=f)
                continue;
            try:
                numVal = float(dose)
            except:
#                 writeline(f,'me parse fail : '+ " : "+str(me))
                print('not parsed: ', me, file=f)
                continue;

            me[2]=numVal;
            
            #discard none value; check again after process
            if(me[2] == None):
#                 writeline(f,'MICRO None value :' + str(me))
                print('not parsed: ', me, file=f)
                continue;
                    
            wholedata.append(['me',(me[0]-admittime).total_seconds(),list(me)])
    return wholedata
            
# processing_microbiologyevents(149416, datetime.datetime.now(), getConnection(), sys.stdout)

## Process prescriptionevents

1. Discard records without starttime.
2. Parse values. Values containing only ','/'<'/'>'/'='/' ' and numbers can be parsed.
3. Discard all none values.
4. Convert units.

In [18]:
def processing_prescriptionevents(aid, admittime, conn, f):
    wholedata = []
    cur = conn.cursor()
    cur.execute('SELECT startdate,formulary_drug_cd,dose_val_rx,dose_unit_rx FROM mimiciii.prescriptions WHERE hadm_id = '+str(aid)+' and formulary_drug_cd in (select * from mengcztemp_itemids_valid_prescript)')
    presevents = cur.fetchall()
    for pe in presevents: 
        #checkdate
        if(pe[0] == None):
#             writeline(f,'pe date 0 : '  + " : "+str(pe))
            print('no starttime: ', pe, file=f)
            continue;
        
        #tuple to list
        pe = list(pe)
        
        if pe[2] is None:
            print('no value: ', pe, file=f)
            continue
        
        # formatting the value
        dose = pe[2]
        dose = dose.replace(',','').replace('<','').replace('>','').replace('=','').replace(' ','')
        numVal = None;
        try:
            numVal = float(dose)
        except:
            if(len(dose.split('-'))==2):
                strs = dose.split('-')
                try:
                    numVal = (float(strs[0]) + float(strs[1]))/2.0
                except:
#                     writeline(f,'pe parse fail : '  + " : "+str(pe))
                    print('not parsed: ', pe, file=f)
                    continue;
            else:
#                 writeline(f,'pe parse fail : '  + " : "+str(pe))
                print('not parsed: ', pe, file=f)
                continue;
        
        pe[2] = numVal;
        
        #discard none value
        if(pe[2] == None):
#             writeline(f,'PRES None value :' + str(pe))
            print('not parsed: ', pe, file=f)
            continue;
            
        #check unit
        # convert units...
        if pe[3] is None:
            pe[3] = ''
        itemid, valuenum, valueuom = pe[1], pe[2], pe[3]
        valueuom = valueuom.replace(' ','').lower()
        mainunit = allitem_unit[map_itemid_index[itemid][0]]
        if valueuom == mainunit or valueuom == '':
            dst_value = float(valuenum)
        else:
            dst_value = None
        pe[2] = dst_value

        #discard none value
        if(pe[2] == None):
#             writeline(f, 'PRES None value :' + str(pe))
            print('not convertible: ', pe, file=f)
            continue;
        
        wholedata.append(['pe',(pe[0]-admittime).total_seconds(),list(pe)])
    return wholedata

processing_prescriptionevents(185777, datetime.datetime.now(), getConnection(), sys.stdout)

[['pe',
  5394041155.550505,
  [datetime.datetime(2191, 3, 16, 0, 0), 'GLAR100I', 6.0, 'UNIT']],
 ['pe',
  5394041155.550505,
  [datetime.datetime(2191, 3, 16, 0, 0), 'BENZ100', 100.0, 'mg']],
 ['pe',
  5394041155.550505,
  [datetime.datetime(2191, 3, 16, 0, 0), 'INSULIN', 0.0, 'UNIT']],
 ['pe',
  5394041155.550505,
  [datetime.datetime(2191, 3, 16, 0, 0), 'GLAR100I', 6.0, 'UNIT']],
 ['pe',
  5394041155.550505,
  [datetime.datetime(2191, 3, 16, 0, 0), 'PRIM263', 52.6, 'mg']],
 ['pe',
  5394041155.550505,
  [datetime.datetime(2191, 3, 16, 0, 0), 'VANC1F', 1000.0, 'mg']],
 ['pe',
  5394041155.550505,
  [datetime.datetime(2191, 3, 16, 0, 0), 'MAGS1I', 1.0, 'gm']],
 ['pe',
  5394041155.550505,
  [datetime.datetime(2191, 3, 16, 0, 0), 'MICROK10', 60.0, 'mEq']],
 ['pe',
  5394041155.550505,
  [datetime.datetime(2191, 3, 16, 0, 0), 'INSULIN', 0.0, 'UNIT']],
 ['pe',
  5394041155.550505,
  [datetime.datetime(2191, 3, 16, 0, 0), 'CLON5', 0.5, 'mg']],
 ['pe',
  5394041155.550505,
  [datetime.date

## Processing

In this step, we generate the matrix of time series for each admission.

1. Discard admissoins without admittime.
2. Collect records from inputevents, outputevents, chartevents, labevents, microbiologyevents and prescriptionevents.
3. For possible conflictions(different values of the same feature occur at the same time):
    1. For numerical values:
        1. For inputevents/outputevents/prescriptions, we use the sum of all conflicting values.
        2. For labevents/chartevents/microbiologyevents, we use the mean of all conflicting values.
    2. For categorical values: we use the value appear first and record that confliction event in the log.
    3. For ratio values: we separate the ratio to two numbers and use the mean of each of them.

In [19]:
# integrate the time series array for a single patient
def processing(aid, f):
    conn = getConnection()
    
    # get admittime
    cur = conn.cursor()
    cur.execute('select admittime from mimiciii.admissions where hadm_id={0}'.format(aid))
    admission = cur.fetchone()
    if admission is None:
        return None
    admittime = admission[0]
    if admittime is None:
        return None
    wholedata = []
    
    # preprocess inputevents
    wholedata.append(processing_inputevents(aid, admittime, conn, f))
    
    # preprocess outputevents
    wholedata.append(processing_outputevents(aid, admittime, conn, f))
    
    # preprocess chartevents
    wholedata.append(processing_chartevents(aid, admittime, conn, f))
    wholedata.append(processing_chartevents_cate(aid, admittime, conn, f))
    wholedata.append(processing_chartevents_num(aid, admittime, conn, f))
    wholedata.append(processing_chartevents_ratio(aid, admittime, conn, f))
    
    # preprocess labevents
    wholedata.append(processing_labevents(aid, admittime, conn, f))
    wholedata.append(processing_labevents_cate(aid, admittime, conn, f))
    wholedata.append(processing_labevents_num(aid, admittime, conn, f))
    wholedata.append(processing_labevents_ratio(aid, admittime, conn, f))
    
    # preprocess microbiologyevents
    wholedata.append(processing_microbiologyevents(aid, admittime, conn, f))
    
    # preprocess prescriptionevents
    wholedata.append(processing_prescriptionevents(aid, admittime, conn, f))
    
    # here is the sparse matrix, order by timestamp
    wholedata = sorted(list(itertools.chain(*wholedata)), key=itemgetter(1))
    
    # transform sparse matrix to matrix
    D = len(allids) + 2
    
    # map time to row
    map_time_index = {}
    index = 0
    for wd in wholedata:
        if(wd[1] not in map_time_index):
            map_time_index[wd[1]] = index
            index += 1
            
    patient = [[None for i in range(D)] for j in range(len(map_time_index))]
    numtodivide = [[0 for i in range(D-2)] for j in range(len(map_time_index))]
#     writeline(f,'len(wholedata) = '+str(len(wholedata)))
#     writeline(f, 'D = '+str(D))
#     writeline(f,'len(patient) = '+str(len(patient)) +' timesteps')

    for wd in wholedata:

        assert patient[ map_time_index[wd[1]] ][D-2] == None or patient[ map_time_index[wd[1]] ][D-2] == wd[1]
        patient[ map_time_index[wd[1]] ][D-2] = wd[1]
        patient[ map_time_index[wd[1]] ][D-1] = aid

        if(wd[0] ==  'ie' or wd[0] ==  'oe' or wd[0] ==  'pe'):
            if(patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]][0]] ==None):
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]][0]] = wd[2][2] 
            else: 
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]][0]] += wd[2][2]

        if(wd[0] ==  'le' or wd[0] == 'ce' or wd[0] == 'me' or wd[0] ==  'lenum' or wd[0] == 'cenum'):
            if wd[2][2] is None:
                print('None value: ', wd, file=f)
            if(patient[map_time_index[wd[1]]] [map_itemid_index[wd[2][1]][0]] == None):
                patient[  map_time_index[wd[1]]  ][  map_itemid_index[wd[2][1]][0]  ] = wd[2][2]
                numtodivide[map_time_index[wd[1]]  ]  [  map_itemid_index[wd[2][1]][0] ] = 1
            else: 
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]][0]] += wd[2][2]
                numtodivide[ map_time_index[wd[1]]  ] [   map_itemid_index[wd[2][1]][0] ] += 1
                
        if (wd[0].startswith('ceratio') or wd[0].startswith('leratio')):
            ot = int(wd[0].split('_')[1]) - 1
            if wd[2][2] is None:
                print(wd, file=f)
            if(patient[map_time_index[wd[1]]] [map_itemid_index[wd[2][1]][ot]] == None):
                patient[  map_time_index[wd[1]]  ][  map_itemid_index[wd[2][1]][ot]  ] = wd[2][2]
                numtodivide[map_time_index[wd[1]]  ]  [  map_itemid_index[wd[2][1]][ot] ] = 1
            else: 
                patient[map_time_index[wd[1]]][map_itemid_index[wd[2][1]][ot]] += wd[2][2]
                numtodivide[ map_time_index[wd[1]]  ] [   map_itemid_index[wd[2][1]][ot] ] += 1
        
        if(wd[0] == 'cecate' or wd[0] == 'lecate'):
            if(patient[map_time_index[wd[1]]] [map_itemid_index[wd[2][1]][0]] == None):
                patient[  map_time_index[wd[1]]  ][  map_itemid_index[wd[2][1]][0]  ] = wd[2][2]
            else: 
                print('DUPLICATED :',wd, file=f)
        
    for i in range(len(map_time_index)):
        for j in range(D-2):
            if(numtodivide[i][j] == 0): continue;
            try:
                patient[i][j] /= numtodivide[i][j]
            except:
                print('div error: ', i, j, file=f)
    conn.close()
    return patient

## Generate information of patient

Here we collect information of one patient, containing its admission_type, admission_location, insurance, language, religion, marital_status and ethnicity.

Since all of them are categorical features, we map the strings of each feature to integers and store the mapping.

In [20]:
# generate general information of patient

# generate map for categorical values
conn = getConnection()
cate = ['admission_type','admission_location','insurance','language','religion','marital_status','ethnicity']
mapping = {}
for c in cate:
    cur = conn.cursor()
    cur.execute('select distinct '+c+' from mimiciii.admissions')
    types = cur.fetchall()
    
    catemapping = {}
    for i in range(len(types)):
        catemapping[types[i][0]] = i 
    mapping[c] = catemapping

# add map for services
cur = conn.cursor()
cur.execute('select distinct ' + 'curr_service' + ' from mimiciii.services')
types = cur.fetchall()

catemapping = {}
for i, typen in enumerate(types):
    catemapping[typen[0]] = i
mapping['curr_service'] = catemapping
# mapping
mapping['curr_service']
np.save('res/adm_catemappings.npy', mapping)
mapping

{'admission_type': {'URGENT': 0, 'NEWBORN': 1, 'ELECTIVE': 2, 'EMERGENCY': 3},
 'admission_location': {'CLINIC REFERRAL/PREMATURE': 0,
  '** INFO NOT AVAILABLE **': 1,
  'TRANSFER FROM HOSP/EXTRAM': 2,
  'EMERGENCY ROOM ADMIT': 3,
  'TRANSFER FROM SKILLED NUR': 4,
  'PHYS REFERRAL/NORMAL DELI': 5,
  'TRANSFER FROM OTHER HEALT': 6,
  'HMO REFERRAL/SICK': 7,
  'TRSF WITHIN THIS FACILITY': 8},
 'insurance': {'Government': 0,
  'Self Pay': 1,
  'Medicare': 2,
  'Private': 3,
  'Medicaid': 4},
 'language': {'SOMA': 0,
  'SERB': 1,
  '**TO': 2,
  '*YOR': 3,
  '*ARM': 4,
  'VIET': 5,
  '*IBO': 6,
  '*PUN': 7,
  '*HUN': 8,
  '*PER': 9,
  'KORE': 10,
  '*KHM': 11,
  'JAPA': 12,
  '*FAR': 13,
  'HAIT': 14,
  None: 15,
  '*BUR': 16,
  'THAI': 17,
  'HIND': 18,
  'CANT': 19,
  'MAND': 20,
  '*CAN': 21,
  '*GUJ': 22,
  'ETHI': 23,
  'GREE': 24,
  '*MAN': 25,
  'ITAL': 26,
  'POLI': 27,
  '*LIT': 28,
  '*CDI': 29,
  '*MOR': 30,
  '*TOI': 31,
  'FREN': 32,
  '*DUT': 33,
  '*BOS': 34,
  '*DEA': 35,
  

## Generate non-temporal features

Here we collect all non-temporal features only related to the admissions:
1. admission id
2. subject id(for finding the patient of one admission)
3. age(at admittime, unit is day)
4. length of stay(unit is minute)
5. in-hospital mortality label
6. labelGurantee label
7. 1-day mortality(from admittime)
8. 2-day mortality(from admittime)
9. 3-day mortality(from admittime)
10. 30-day mortality(from dischtime)
11. 1-year mortality(from dischtime)
12. admission_type
13. admission_location
14. insurance
15. language
16. religion
17. marital_status
18. ethnicity

**Mortality label here is not used, please refer to 8_collect_time_labels.ipynb to get correct mortality labels. We leave them here only for compatibility.**

In [21]:
def ageLosMortality(aid, f):
    conn = getConnection()
    
    cur = conn.cursor()
    cur.execute('SELECT hadm_id,subject_id,admittime,dischtime,deathtime,admission_type,admission_location,insurance,language,religion,marital_status,ethnicity FROM mimiciii.ADMISSIONS WHERE hadm_id='+str(aid))
    admission = cur.fetchone()
    
    assert admission!=None
    
    subject_id = admission[1]
    admittime = admission[2]
    dischtime = admission[3]
    deathtime = admission[4]
    
    cur = conn.cursor()
    cur.execute('SELECT dob, dod FROM mimiciii.PATIENTS WHERE subject_id='+str(subject_id))
    patient = cur.fetchone()

    assert patient!=None
    birthdate = patient[0]
    final_deathdate = patient[1]
    mortal = 0
    labelGuarantee = 0
    die24 = 0
    die24_48 = 0
    die48_72 = 0
    die30days = 0
    die1year = 0
    if(deathtime != None) : 
        mortal = 1
        if(deathtime != dischtime):
            labelGuarantee=1
        secnum = (deathtime - admittime).total_seconds()
        if secnum <= 24 * 60 * 60:
            die24 = 1
        if secnum <= 48 * 60 * 60:
            die24_48 = 1
        if secnum <= 72 * 60 * 60:
            die48_72 = 1
    if dischtime is not None and final_deathdate is not None:
        dischsecnum = (final_deathdate - dischtime).total_seconds()
        if dischsecnum <= 30 * 24 * 60 * 60:
            die30days = 1
        if dischsecnum <= 365 * 24 * 60 * 60:
            die1year = 1
            
    cur.execute('select curr_service from mimiciii.services where hadm_id='+str(aid))
    curr_service = cur.fetchone()
    if curr_service:
        curr_service = curr_service[0]
    else:
        curr_service = 'NB'

    data = [aid,subject_id, (admittime - birthdate).total_seconds()/(3600*24), (dischtime-admittime).total_seconds()//60., mortal, labelGuarantee, die24, die24_48, die48_72, die30days, die1year, mapping['curr_service'][curr_service]]
    for i in range(5,12):
        data.append(mapping[cate[i-5]][admission[i]])
    conn.close()
    return data

# ageLosMortality(128652, sys.stdout)

## Generate ICD9 codes

Here we convert icd9 codes to category numbers.

In [22]:
def ICD9(aid, f):
    conn = getConnection()
    cate20=0
    
    cur = conn.cursor()
    cur.execute('SELECT icd9_code FROM mimiciii.DIAGNOSES_ICD WHERE hadm_id='+str(aid)+' ORDER BY seq_num')
    icd9s = cur.fetchall() 
    list_icd9 = []
    for icd9 in icd9s:
        icd = icd9[0];
        if icd is None:
            continue
        if(icd[0] =='V'):
            label_name = 19
            numstr = icd[0:3]+'.'+icd[3:len(icd)]
        elif(icd[0] =='E'):
            cate20 +=1
            label_name = 20
            numstr = icd
        else:
            num = float(icd[0:3])
            numstr = icd[0:3]+'.'+icd[3:len(icd)]
            if(num >= 1 and num <= 139 ) :
                label_name = 0
            if(num >= 140 and num <= 239 ) :
                label_name = 1
            if(num >= 240 and num <= 279 ) :
                label_name = 2
            if(num >= 280 and num <= 289 ) :
                label_name = 3
            if(num >= 290 and num <= 319 ) :
                label_name = 4
            if(num >= 320 and num <= 389 ) :
                label_name = 5
            if(num >= 390 and num <= 459 ) :
                label_name = 6
            if(num >= 460 and num <= 519 ) :
                label_name = 7
            if(num >= 520 and num <= 579 ) :
                label_name = 8
            if(num >= 580 and num <= 629 ) :
                label_name = 9
            if(num >= 630 and num <= 677 ) :
                label_name = 10
            if(num >= 680 and num <= 709 ) :
                label_name = 11
            if(num >= 710 and num <= 739 ) :
                label_name = 12
            if(num >= 740 and num <= 759 ) :
                label_name = 13
            if(num >= 760 and num <= 779 ) :
                label_name = 14
            if(num >= 780 and num <= 789 ) :
                label_name = 15
            if(num >= 790 and num <= 796 ) :
                label_name = 16
            if(num >= 797 and num <= 799 ) :
                label_name = 17
            if(num >= 800 and num <= 999 ) :
                label_name = 18
        list_icd9.append([aid,icd,numstr,label_name])
    conn.close()
    return list_icd9

# ICD9(185777, sys.stdout)

In [23]:
if os.path.exists('admdata/'):
    shutil.rmtree('admdata/')
os.makedirs('admdata/log/')

## Save one file for each admission

For each admission, we save a separate file for it, which contains:
1. 'timeseries': matrix of time series in form of sparse matrix
2. 'general': non-temporal features
3. 'icd9': list of icd9 category codes

In [24]:
def process_patient(aid):
    with open('admdata/log/adm-{0}.log'.format(str('%.6d' % aid)), 'w') as f:
        try:
            proc = processing(aid, f)
            if len(proc) == 0:
                return
            res = {
                'timeseries': sparsify(proc),
                'general': ageLosMortality(aid, f),
                'icd9': ICD9(aid, f)
            }
            np.save('admdata/adm-' + str('%.6d' % aid), res)
            print('finished {0}!'.format(aid))
        except Exception as e:
            with open('admdata/log/admerror-{0}.log'.format(str('%.6d' % aid)), 'w') as ferr:
                traceback.print_exc(file=ferr)
            traceback.print_exc(sys.stdout)
            print('failed at {0}!'.format(aid))
    
process_patient(136796)

finished 136796!


In [25]:
#p = Pool(10)
for aid in admission_ids:
    #p.apply_async(process_patient, args=(aid,))
    process_patient(aid,)
#p.close()
#p.join()

finished 155703!
finished 130406!
finished 194150!
finished 117336!
finished 106095!
finished 167276!
finished 146688!
finished 103729!
finished 109863!
finished 139056!
finished 174416!
finished 180028!
finished 129353!
finished 196916!
finished 165533!
finished 164125!
finished 196633!
finished 159965!
finished 103155!
finished 103443!
finished 130544!
finished 166036!
finished 191028!
finished 129748!
finished 160242!
finished 192635!
finished 191053!
finished 141968!
finished 131290!
finished 155892!
finished 180838!
finished 126725!
finished 125506!
finished 159265!
finished 108686!
finished 132176!
finished 115756!
finished 162767!
finished 166390!
finished 108457!
finished 181223!
finished 114581!
finished 102859!
finished 169150!
finished 134370!
finished 116016!
finished 164383!
finished 106912!
finished 190359!
finished 106115!
finished 167981!
finished 160849!
finished 111919!
finished 103013!
finished 106137!
finished 156884!
finished 145284!
finished 198654!
finished 13408

finished 101477!
finished 118251!
finished 140703!
finished 147253!
finished 165133!
finished 112949!
finished 132888!
finished 149504!
finished 172076!
finished 175238!
finished 101703!
finished 174646!
finished 122926!
finished 188732!
finished 174698!
finished 131483!
finished 177253!
finished 104466!
finished 184590!
finished 147022!
finished 115665!
finished 138940!
finished 127886!
finished 163928!
finished 100543!
finished 184278!
finished 196915!
finished 103280!
finished 155208!
finished 111648!
finished 155348!
finished 123962!
finished 193211!
finished 152792!
finished 177053!
finished 140652!
finished 189247!
finished 127601!
finished 126109!
finished 128924!
finished 158788!
finished 152232!
finished 167544!
finished 124218!
finished 139218!
finished 161115!
finished 186095!
finished 115126!
finished 184294!
finished 119167!
finished 121201!
finished 139440!
finished 156275!
finished 184807!
finished 123117!
finished 103750!
finished 172482!
finished 194481!
finished 14025

finished 198504!
finished 181231!
finished 110693!
finished 149686!
finished 172384!
finished 182889!
finished 158720!
finished 127040!
finished 161456!
finished 181632!
finished 141877!
finished 168138!
finished 150868!
finished 108704!
finished 138706!
finished 116578!
finished 170645!
finished 166575!
finished 131145!
finished 194372!
finished 106093!
finished 135269!
finished 196824!
finished 104667!
finished 163820!
finished 107093!
finished 144626!
finished 157546!
finished 176622!
finished 169440!
finished 173741!
finished 157998!
finished 121354!
finished 185156!
finished 128389!
finished 163829!
finished 114872!
finished 146103!
finished 115481!
finished 111012!
finished 194017!
finished 191203!
finished 153982!
finished 188377!
finished 153131!
finished 103499!
finished 141229!
finished 142320!
finished 102291!
finished 121516!
finished 150592!
finished 153790!
finished 133353!
finished 192314!
finished 124154!
finished 110711!
finished 189126!
finished 124519!
finished 13475

finished 157274!
finished 170082!
finished 159703!
finished 133086!
finished 171769!
finished 112391!
finished 178025!
finished 101715!
finished 186879!
finished 103295!
finished 185402!
finished 191375!
finished 165066!
finished 115664!
finished 134380!
finished 130094!
finished 162712!
finished 143579!
finished 189276!
finished 172867!
finished 130232!
finished 125438!
finished 166912!
finished 186401!
finished 117495!
finished 151281!
finished 106546!
finished 163631!
finished 155588!
finished 141898!
finished 194332!
finished 166226!
finished 102492!
finished 127853!
finished 125104!
finished 114688!
finished 113028!
finished 100735!
finished 127410!
finished 115121!
finished 176285!
finished 173358!
finished 182212!
finished 123087!
finished 109429!
finished 181429!
finished 128596!
finished 105324!
finished 169184!
finished 154127!
finished 126904!
finished 181057!
finished 117645!
finished 175520!
finished 119257!
finished 126971!
finished 185833!
finished 195349!
finished 10184

finished 184298!
finished 135932!
finished 150720!
finished 133457!
finished 153878!
finished 152345!
finished 150762!
finished 179525!
finished 151888!
finished 191448!
finished 136025!
finished 148229!
finished 188114!
finished 170483!
finished 141507!
finished 192249!
finished 127770!
finished 130518!
finished 192639!
finished 159604!
finished 195683!
finished 186139!
finished 176586!
finished 130294!
finished 131712!
finished 173534!
finished 131018!
finished 109204!
finished 187278!
finished 127699!
finished 131977!
finished 168691!
finished 122676!
finished 161540!
finished 130020!
finished 129921!
finished 131114!
finished 189277!
finished 130166!
finished 163210!
finished 195997!
finished 131357!
finished 131246!
finished 167836!
finished 117291!
finished 104181!
finished 158293!
finished 168146!
finished 101087!
finished 126082!
finished 172269!
finished 151807!
finished 194146!
finished 166487!
finished 149271!
finished 151178!
finished 175950!
finished 106238!
finished 16604

...

finished 162381!
finished 157381!
finished 124830!
finished 114700!
finished 163306!
finished 120823!
finished 114802!
finished 139951!
finished 171997!
finished 140228!
finished 115354!
finished 119214!
finished 137277!
finished 163643!
finished 106600!
finished 128980!
finished 119700!
finished 155956!
finished 158547!
finished 137099!
finished 109012!
finished 111945!
finished 183575!
finished 152390!
finished 125085!
finished 113010!
finished 175196!
finished 129928!
finished 180315!
finished 179497!
finished 195936!
finished 178005!
finished 139732!
finished 176552!
finished 196346!
finished 103104!
finished 121568!
finished 146215!
finished 103829!
finished 153645!
finished 151705!
finished 137644!
finished 152970!
finished 100257!
finished 189969!
finished 138330!
finished 175436!
finished 177125!
finished 116935!
finished 159833!
finished 178185!
finished 158344!
finished 134385!
finished 103140!
finished 194613!
finished 147234!
finished 174153!
finished 172999!
finished 14574

finished 186000!
finished 185410!
finished 192298!
finished 181055!
finished 152538!
finished 149510!
finished 184103!
finished 143652!
finished 186724!
finished 188564!
finished 155723!
finished 130380!
finished 163440!
finished 114502!
finished 154589!
finished 189567!
finished 139446!
finished 107371!
finished 180356!
finished 175702!
finished 191493!
finished 180082!
finished 138509!
finished 164211!
finished 136861!
finished 109056!
finished 180162!
finished 165686!
finished 156074!
finished 117741!
finished 109162!
finished 189423!
finished 117838!
finished 185902!
finished 193799!
finished 163565!
finished 171974!
finished 172254!
finished 168673!
finished 180900!
finished 108710!
finished 144694!
finished 110066!
finished 133126!
finished 186208!
finished 123144!
finished 102032!
finished 183975!
finished 188698!
finished 123853!
finished 176457!
finished 168776!
finished 128868!
finished 166543!
finished 111208!
finished 185532!
finished 197452!
finished 191991!
finished 12455

finished 193744!
finished 135216!
finished 175038!
finished 155881!
finished 170098!
finished 152937!
finished 121868!
finished 149059!
finished 109517!
finished 147537!
finished 189923!
finished 186196!
finished 150276!
finished 118496!
finished 175869!
finished 155505!
finished 185088!
finished 157570!
finished 154333!
finished 171993!
finished 101864!
finished 193147!
finished 145839!
finished 107527!
finished 136619!
finished 108698!
finished 115419!
finished 140145!
finished 177998!
finished 118259!
finished 193895!
finished 113137!
finished 124454!
finished 101328!
finished 101405!
finished 109201!
finished 173710!
finished 122530!
finished 134655!
finished 139389!
finished 192146!
finished 119266!
finished 151042!
finished 100852!
finished 180550!
finished 145020!
finished 156857!
finished 191334!
finished 148067!
finished 117289!
finished 130231!
finished 189118!
finished 125393!
finished 187269!
finished 124761!
finished 156914!
finished 169080!
finished 120726!
finished 17583

finished 103517!
finished 140926!
finished 101805!
finished 169132!
finished 150589!
finished 106333!
finished 196755!
finished 169622!
finished 141219!
finished 130765!
finished 195476!
finished 113169!
finished 160982!
finished 125187!
finished 151351!
finished 135238!
finished 104205!
finished 123234!
finished 169958!
finished 115028!
finished 141061!
finished 103618!
finished 109837!
finished 117650!
finished 142346!
finished 154032!
finished 153353!
finished 153248!
finished 174265!
finished 197621!
finished 176559!
finished 154445!
finished 150105!
finished 153526!
finished 127195!
finished 103995!
finished 163886!
finished 187284!
finished 174906!
finished 151286!
finished 142207!
finished 146581!
finished 135260!
finished 174257!
finished 135989!
finished 141600!
finished 163844!
finished 144846!
finished 179423!
finished 163626!
finished 111818!
finished 115996!
finished 122925!
finished 175146!
finished 130215!
finished 173020!
finished 196345!
finished 170440!
finished 12726

finished 169636!
finished 176225!
finished 192383!
finished 198066!
finished 161204!
finished 138259!
finished 120230!
finished 146730!
finished 160091!
finished 147371!
finished 131148!
finished 195991!
finished 152185!
finished 153199!
finished 139826!
finished 196313!
finished 114613!
finished 102051!
finished 194346!
finished 164698!
finished 122250!
finished 129232!
finished 195747!
finished 116231!
finished 147516!
finished 135066!
finished 161329!
finished 131784!
finished 195500!
finished 153572!
finished 177696!
finished 102511!
finished 145397!
finished 136599!
finished 193790!
finished 189830!
finished 177244!
finished 139269!
finished 199956!
finished 112817!
finished 154986!
finished 160251!
finished 147249!
finished 159730!
finished 146240!
finished 196305!
finished 106489!
finished 172359!
finished 168307!
finished 112397!
finished 116643!
finished 158758!
finished 178465!
finished 139952!
finished 155292!
finished 190627!
finished 172891!
finished 107809!
finished 12386

In [26]:
# add labels about mortality, now we have 1day|2days|3days|in-hospitial|30days|1year
def add_mortality_labels(aid):
    with open('admdata/log/adm-addml{0}.log'.format(str('%.6d' % aid)), 'w') as f:
        try:
            res = np.load('admdata/adm-' + str('%.6d' % aid)+'.npy').tolist()
        except Exception as e:
            traceback.print_exc(file=f)
            return
        res['general'] = ageLosMortality(aid, f)
        np.save('admdata/adm-' + str('%.6d' % aid)+'.npy', res)
    print('finished {0}!'.format(aid))

#p = Pool()
for aid in admission_ids:
    #p.apply_async(add_mortality_labels, args=(aid,))
    add_mortality_labels(aid,)
#p.close()
#p.join()
# t = 0
# for aid in admission_ids:
#     add_mortality_labels(aid, sys.stdout)
#     t += 1
#     if t % 100 == 0:
#         print(t)

finished 155703!
finished 130406!
finished 194150!
finished 117336!
finished 106095!
finished 167276!
finished 146688!
finished 103729!
finished 109863!
finished 139056!
finished 174416!
finished 180028!
finished 129353!
finished 196916!
finished 165533!
finished 164125!
finished 196633!
finished 159965!
finished 103155!
finished 103443!
finished 130544!
finished 166036!
finished 191028!
finished 129748!
finished 160242!
finished 192635!
finished 191053!
finished 141968!
finished 131290!
finished 155892!
finished 180838!
finished 126725!
finished 125506!
finished 159265!
finished 108686!
finished 132176!
finished 115756!
finished 162767!
finished 166390!
finished 108457!
finished 181223!
finished 114581!
finished 102859!
finished 169150!
finished 134370!
finished 116016!
finished 164383!
finished 106912!
finished 190359!
finished 106115!
finished 167981!
finished 160849!
finished 111919!
finished 103013!
finished 106137!
finished 156884!
finished 145284!
finished 198654!
finished 13408

finished 140703!
finished 147253!
finished 165133!
finished 112949!
finished 132888!
finished 149504!
finished 172076!
finished 175238!
finished 101703!
finished 174646!
finished 122926!
finished 188732!
finished 174698!
finished 131483!
finished 177253!
finished 104466!
finished 184590!
finished 147022!
finished 115665!
finished 138940!
finished 127886!
finished 163928!
finished 100543!
finished 184278!
finished 196915!
finished 103280!
finished 155208!
finished 111648!
finished 155348!
finished 123962!
finished 193211!
finished 152792!
finished 177053!
finished 140652!
finished 189247!
finished 127601!
finished 126109!
finished 128924!
finished 158788!
finished 152232!
finished 167544!
finished 124218!
finished 139218!
finished 161115!
finished 186095!
finished 115126!
finished 184294!
finished 119167!
finished 121201!
finished 139440!
finished 156275!
finished 184807!
finished 123117!
finished 103750!
finished 172482!
finished 194481!
finished 140252!
finished 101768!
finished 16259

finished 110693!
finished 149686!
finished 172384!
finished 182889!
finished 158720!
finished 127040!
finished 161456!
finished 181632!
finished 141877!
finished 168138!
finished 150868!
finished 108704!
finished 138706!
finished 116578!
finished 170645!
finished 166575!
finished 131145!
finished 194372!
finished 106093!
finished 135269!
finished 196824!
finished 104667!
finished 163820!
finished 107093!
finished 144626!
finished 157546!
finished 176622!
finished 169440!
finished 173741!
finished 157998!
finished 121354!
finished 185156!
finished 128389!
finished 163829!
finished 114872!
finished 146103!
finished 115481!
finished 111012!
finished 194017!
finished 191203!
finished 153982!
finished 188377!
finished 153131!
finished 103499!
finished 141229!
finished 142320!
finished 102291!
finished 121516!
finished 150592!
finished 153790!
finished 133353!
finished 192314!
finished 124154!
finished 110711!
finished 189126!
finished 124519!
finished 134758!
finished 181325!
finished 13427

finished 159703!
finished 133086!
finished 171769!
finished 112391!
finished 178025!
finished 101715!
finished 186879!
finished 103295!
finished 185402!
finished 191375!
finished 165066!
finished 115664!
finished 134380!
finished 130094!
finished 162712!
finished 143579!
finished 189276!
finished 172867!
finished 130232!
finished 125438!
finished 166912!
finished 186401!
finished 117495!
finished 151281!
finished 106546!
finished 163631!
finished 155588!
finished 141898!
finished 194332!
finished 166226!
finished 102492!
finished 127853!
finished 125104!
finished 114688!
finished 113028!
finished 100735!
finished 127410!
finished 115121!
finished 176285!
finished 173358!
finished 182212!
finished 123087!
finished 109429!
finished 181429!
finished 128596!
finished 105324!
finished 169184!
finished 154127!
finished 126904!
finished 181057!
finished 117645!
finished 175520!
finished 119257!
finished 126971!
finished 185833!
finished 195349!
finished 101841!
finished 167603!
finished 12676

finished 153878!
finished 152345!
finished 150762!
finished 179525!
finished 151888!
finished 191448!
finished 136025!
finished 148229!
finished 188114!
finished 170483!
finished 141507!
finished 192249!
finished 127770!
finished 130518!
finished 192639!
finished 159604!
finished 195683!
finished 186139!
finished 176586!
finished 130294!
finished 131712!
finished 173534!
finished 131018!
finished 109204!
finished 187278!
finished 127699!
finished 131977!
finished 168691!
finished 122676!
finished 161540!
finished 130020!
finished 129921!
finished 131114!
finished 189277!
finished 130166!
finished 163210!
finished 195997!
finished 131357!
finished 131246!
finished 167836!
finished 117291!
finished 104181!
finished 158293!
finished 168146!
finished 101087!
finished 126082!
finished 172269!
finished 151807!
finished 194146!
finished 166487!
finished 149271!
finished 151178!
finished 175950!
finished 106238!
finished 166046!
finished 182117!
finished 171772!
finished 108874!
finished 10908

...

finished 114128!
finished 141920!
finished 140488!
finished 198562!
finished 144394!
finished 146921!
finished 172091!
finished 134788!
finished 144234!
finished 190185!
finished 176076!
finished 187470!
finished 167500!
finished 169035!
finished 169607!
finished 145027!
finished 108285!
finished 171703!
finished 102981!
finished 114329!
finished 186586!
finished 120636!
finished 131593!
finished 158470!
finished 147320!
finished 150628!
finished 120213!
finished 109016!
finished 156435!
finished 115451!
finished 136122!
finished 111640!
finished 113191!
finished 181702!
finished 178575!
finished 185767!
finished 188657!
finished 116726!
finished 103119!
finished 127072!
finished 124945!
finished 137013!
finished 197351!
finished 119648!
finished 165780!
finished 160603!
finished 142057!
finished 101818!
finished 179787!
finished 177439!
finished 198962!
finished 198626!
finished 103662!
finished 111609!
finished 124229!
finished 101052!
finished 154115!
finished 120800!
finished 18039

finished 152763!
finished 136022!
finished 140091!
finished 148919!
finished 172344!
finished 118704!
finished 151323!
finished 114287!
finished 120892!
finished 107787!
finished 108363!
finished 169468!
finished 193501!
finished 111104!
finished 157518!
finished 194022!
finished 130917!
finished 191759!
finished 116936!
finished 123845!
finished 166901!
finished 190943!
finished 144865!
finished 157637!
finished 163281!
finished 150427!
finished 124878!
finished 140851!
finished 181636!
finished 191645!
finished 187967!
finished 186483!
finished 146000!
finished 143169!
finished 137570!
finished 187081!
finished 174660!
finished 162881!
finished 115853!
finished 104701!
finished 154438!
finished 133456!
finished 162299!
finished 169330!
finished 172829!
finished 186618!
finished 118508!
finished 154364!
finished 188786!
finished 172921!
finished 109826!
finished 191631!
finished 157512!
finished 139364!
finished 146991!
finished 178045!
finished 163893!
finished 174748!
finished 13703

finished 171166!
finished 153100!
finished 117308!
finished 185502!
finished 153365!
finished 147851!
finished 167482!
finished 121845!
finished 188782!
finished 172473!
finished 147080!
finished 120138!
finished 150751!
finished 191259!
finished 178598!
finished 175372!
finished 120640!
finished 103903!
finished 193866!
finished 176515!
finished 175092!
finished 154591!
finished 153204!
finished 198082!
finished 189819!
finished 120864!
finished 114914!
finished 111887!
finished 101513!
finished 114030!
finished 198590!
finished 100674!
finished 127412!
finished 100490!
finished 102014!
finished 124610!
finished 169470!
finished 143430!
finished 102664!
finished 194819!
finished 128532!
finished 166743!
finished 173321!
finished 193030!
finished 123872!
finished 182315!
finished 144190!
finished 147376!
finished 108352!
finished 195309!
finished 169775!
finished 170279!
finished 108585!
finished 131447!
finished 146015!
finished 143670!
finished 177350!
finished 125179!
finished 10941

finished 111611!
finished 129380!
finished 199993!
finished 195029!
finished 115468!
finished 151858!
finished 182458!
finished 174857!
finished 183101!
finished 135719!
finished 193198!
finished 161992!
finished 179896!
finished 159838!
finished 139099!
finished 183187!
finished 189846!
finished 111170!
finished 162322!
finished 149349!
finished 146789!
finished 164751!
finished 151725!
finished 149793!
finished 129020!
finished 115786!
finished 174395!
finished 168232!
finished 183107!
finished 115967!
finished 179568!
finished 145740!
finished 152550!
finished 100427!
finished 120811!
finished 100897!
finished 146315!
finished 125550!
finished 162365!
finished 154583!
finished 149298!
finished 110591!
finished 164155!
finished 147623!
finished 155922!
finished 130010!
finished 152923!
finished 135962!
finished 181542!
finished 102809!
finished 102151!
finished 159757!
finished 139476!
finished 136289!
finished 172491!
finished 167451!
finished 138775!
finished 167381!
finished 17072

finished 173868!
finished 106209!
finished 172786!
finished 165433!
finished 130002!
finished 162522!
finished 162285!
finished 199047!
finished 178109!
finished 137008!
finished 113896!
finished 196874!
finished 144680!
finished 171006!
finished 155847!
finished 124199!
finished 155243!
finished 131270!
finished 116589!
finished 124912!
finished 137802!
finished 159835!
finished 183153!
finished 195518!
finished 156068!
finished 120932!
finished 122695!
finished 182577!
finished 133964!
finished 150258!
finished 199392!
finished 150889!
finished 156451!
finished 149846!
finished 162814!
finished 177198!
finished 160748!
finished 148471!
finished 116190!
finished 146618!
finished 134436!
finished 135245!
finished 131992!
finished 141023!
finished 184328!
finished 139349!
finished 108851!
finished 179561!
finished 182003!
finished 196649!
finished 143112!
finished 177843!
finished 138791!
finished 161505!
finished 174587!
finished 181008!
finished 154411!
finished 120110!
finished 14647

In [27]:
def copy_first_admissions(aid):
    if aid in admission_first_ids_set:
        res = np.load(os.path.join('admdata', 'adm-%.6d.npy' % aid))
        np.save(os.path.join('admdata_first', 'adm-%.6d.npy' % aid), res)
        
#p = Pool()
#p.map(copy_first_admissions, admission_ids)
copy_first_admissions(admission_ids)
#p.close()
#p.join()

NameError: name 'admission_first_ids_set' is not defined